## Environment


In [1]:
import os
# Environment define
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
os.environ["OPENAI_BASE_URL"] = "https://api.openai.com/v1"


## Load Pdf

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Data/GRI 306_ Waste 2020.pdf")
pages = loader.load_and_split()
print(len(pages))

31


## Prompt Define

In [8]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

# Define the structure for Indicators
class IndicatorTriples(BaseModel):
    """Identifying information about indicators."""
    indicators: List[str] = Field(
        description="A sustainability reporting indicator is defined as 'a single measurement from which quantitative conclusions on the environmental phenomenon can be inferred'. "
                    "If the provided text does not contain specific sustainability reporting indicators as defined by the task, return: None."
    )
    triples: List[str] = Field(
        description="Extract triples (subject, predicate, object)."
    )

# Define the prompt template
system= """You are a knowledge graph engineer tasked with extracting triples (subject, predicate, object) from a sustainability reporting standard text to populate a knowledge graph. 

Focus exclusively on sustainability reporting indicators. 
Definition: a single quantity of measure from which quantitative conclusions on the environmental phenomenon can be inferred. 

### Indication ###
There are 4 kinds of predicates that can be used for extracting triples: 
1. Has quantity kind: Relates the indicator to the measurement quantity kind. 
    - A Quantity Kind is any observable property that can be measured and quantified numerically. 
    - Familiar examples include physical properties such as length, mass, time, force, energy, power, electric charge, etc. 
    - Less familiar examples include currency, interest rate, price-to-earning ratio, and information capacity. 
    - EACH INDICATOR HAS ONLY ONE SINGLE QUANTITY KIND.
2. Has measurement phenomenon: Relates the indicator to the environmental phenomenon measured. 
3. Has unit: Relates the indicator to the required unit derived from sustainability standards. 
    - A unit is a particular quantity value that has been chosen as a scale for measuring other quantities of the same kind (more generally of equivalent dimension). 
    - For example, the meter is a quantity of length that has been rigorously defined and standardized by the BIPM (International Board of Weights and Measures). 
    - THIS PROPERTY IS OPTIONAL. SOMETIMES THE REQUIRED UNITS ARE SPECIFIED IN THE STANDARD, BUT SOMETIMES NOT. 
4. Has applicability: Relates the indicator to its applicability status, which can be either optional (indicated by 'if applicable') or mandatory (indicated by 'shall'). 

There are some requirements for extracted indicators and triples: 
1. If the provided text does not contain specific sustainability reporting indicators as defined by the task, return: None. 
2. The extracted indicators do not need a period(comma). 

### Example ###
Here are some examples about extracting indicators and types of triples: 
    example_user: Extract indicators and triples from the following text: "The reporting organization shall report the following information:  a. Total weight of waste generated in metric tons, and a breakdown of this total by composition of the waste. b. Contextual information necessary to understand the data and how the data has been compiled." 
    example_assistant: {{"indicators": ["Total weight of waste generated in metric tons"], "triples": ["(Total weight of waste generated in metric tons, has quantity kind, weight)", "(Total weight of waste generated in metric tons, has measurement phenomenon, waste generated)","(Total weight of waste generated in metric tons, has unit, metric tons)", "(Total weight of waste generated in metric tons, has applicability, mandatory)"]}} """

# Create a ChatPromptTemplate instance
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

# Initialize the LLM with structured output
model = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)
structured_llm = model.with_structured_output(IndicatorTriples)

# Combine the prompt and the structured LLM to create a chain
prompt_structured_llm = prompt | structured_llm

## Chain

In [9]:
import json

# Save path
filename = "./output/triples_GRI_306.json"
# Initialize the dictionary to store the triples
triples_dict = {}

# Process each page and extract indicators and triples
for i, page_text in enumerate(pages):
    result = prompt_structured_llm.invoke({"input": page_text})
    triples_dict[f"Page_{i}"] = result.dict()  # Directly storing the dictionary

# Convert results to JSON format
with open(filename, 'w') as file:
    json.dump(triples_dict, file, indent=4)

print(f"Results written to {filename}")


Results written to ./output/triples_GRI_306.json
